In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import concurrent.futures 
from fuzzywuzzy import fuzz, process
from difflib import SequenceMatcher
from os import walk
from bs4 import BeautifulSoup
import requests
import time
import random
pd.set_option('display.max_columns', 100)
import statistics

In [2]:
def get_winner(url):
    try:
        page = requests.get(url)
        time.sleep(random.randint(0, 3))
        soup = BeautifulSoup(page.content, 'html.parser')
        score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
        score = int(score[0]) - int(score[1])
        score = winner(score)
        return score
    except:
        return 'remove'

In [3]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [4]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [6]:
o = 'C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'
n = 'C:/Users\idotz/Desktop\Buffalo Bets/data'
old_f = next(walk(o), (None, None, []))[2]
old_f = [f'{o}/{f}' for f in old_f if '2022' in f and '_data' in f and 'dict' not in f and 'all' not in f]
new_f = next(walk(n), (None, None, []))[2]
new_f = [f'{n}/{f}' for f in new_f]
file_names = old_f + new_f

In [7]:
all_data = []
for f in file_names:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

In [8]:
len(all_data)

1583

In [9]:
res = []
for idx, dic in enumerate(all_data):
    try:
        e_dict = {}
        for name, tup in dic.items():
            name = name.replace(' ','_').lower()
            try:
                if (len(tup) == 2) and (isinstance(tup, list)) and (isinstance(tup[0], str) or 
                                                               isinstance(tup[0], float)):
                    e_dict[f'{name}_diff'] = float(tup[0]) - float(tup[1])
                else:
                    e_dict[name] = tup
            except:
                e_dict[name] = tup        
        if 'bet' in e_dict:
            e_dict['odds_sum'] = e_dict['bet']['odds_checker_sum']
            del e_dict['bet']
        e_dict['median_odds'] = e_dict.pop('odds')
        if len(e_dict['median_odds']) == 3:
            e_dict['std_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                          [np.std(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            e_dict['75_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                          [np.percentile(sorted(x), 75) for x in e_dict['full_odds']]))
#         e_dict['cv_odds'] = np.std(list(e_dict['median_odds'].values()))/np.mean(list(e_dict['median_odds'].values()))
#         elif ((len(e_dict['median_odds']) != 3) & (similar("".join(sorted(e_dict['match_name'])), "".join(sorted(e_dict['api_odds']['best_match']))) > 0.9)
#            & ('api_odds' in e_dict)):
#             odds = [i for j in list(e_dict['api_odds'].values())[:-1] for i in j]
#             odds = np.array(odds).T.tolist()
#             names = e_dict['match_name'].split('-vs-')
#             names = [names[0]] + ['draw'] + [names[1]]
#             e_dict['median_odds'] = dict(zip(names, [statistics.median(o[1:-1]) for o in odds]))
#             e_dict['std_odds'] = dict(zip(names, [np.std(sorted(x)[1:-1]) for x in odds]))
#             odds_sum = np.sum([1/o for o in e_dict['median_odds'].values()])
#             e_dict['odds_sum'] = odds_sum
#         else:
#             pass
        res.append(e_dict)
    except Exception as e:
        res.append(dic)

C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
len(res)

1583

In [11]:
df = pd.DataFrame(res)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['median_odds'] != '') & (df['median_odds'] != {}) & (df['median_odds'].notna())]
df = df[df['median_odds'].map(len) == 3]
df = df[(df['odds_sum'] > 1.03) & (df['odds_sum'] < 1.11)]
df['full_odds_min'] = [min([len(x) for x in l]) for l in df['full_odds']]
df = df[df['full_odds_min'] > 8]
df = df.reset_index(drop = True)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
for name in ['std','median']:
    r = []
    for i in df[[f'{name}_odds','o_names']].to_numpy():
        match = [getname(name, list(i[0].keys())) for name in i[1]]
        match = [i[0][key] for key in match]
        r.append(match)
    df[f'{name}_odds'] = r
df['odds_predict'] = df['median_odds'].apply(lambda x: np.argmin(x))
ordo_df = pd.DataFrame(df['median_odds'].tolist(), columns=['med_0', 'med_1', 'med_2'])
std_o = pd.DataFrame(df['std_odds'].tolist(), columns=['std_0', 'std_1', 'std_2'])
# sevfiv_o = pd.DataFrame(df['75_odds'].tolist(), columns=['75_0', '75_1', '75_2'])
df = pd.concat([df, std_o, ordo_df], axis=1)
# df[['med_odds_prob_0','med_odds_prob_1','med_odds_prob_2']] = df[['med_0','med_1','med_2']].apply(lambda x: 1/x)
# df[['75_odds_prob_0','75_odds_prob_1','75_odds_prob_2']] = df[['75_0','75_1','75_2']].apply(lambda x: 1/x)
df['cv_odds'] = df[['med_0','med_1','med_2']].std(axis = 1)/df[['med_0','med_1','med_2']].mean(axis = 1)
df = df.drop(['half_time','o_names', '75_odds', 'median_odds',
             'std_odds', 'full_odds','match_name', 'full_odds_min','league_name',
             'xg_first_half_diff','xg_second_half_diff','api_odds'], axis = 1)
print(df.shape)

(1175, 55)


In [12]:
remove = [812, 232, 814, 300, 19, 293, 64]

In [13]:
test = df[['url','odds_url']]
test['url'] = test['url'].apply(lambda x: x.split('/')[-1])
test['odds_url'] = test['odds_url'].apply(lambda x: x.split('/')[-2])
test['s'] = test[['url','odds_url']].apply(lambda x: similar(x.url,x.odds_url), axis = 1)
test = test.sort_values(by = 's')
test.head(50)

C:\Users\idotz\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,url,odds_url,s
812,ifk-norrk%C3%B6ping-vs-mj%C3%A4llby,ifk-varnamo-v-kalmar-ff,0.344828
232,bayern-m%C3%BCnchen-vs-vfb-stuttgart,augsburg-v-bayern-munich,0.366667
814,trabzonspor-vs-kas%C4%B1mpa%C5%9Fa,besiktas-v-trabzonspor,0.392857
300,kristiansund-vs-v%C3%A5lerenga,aalesund-v-sarpsborg,0.400000
19,cambuur-vs-az-alkmaar,fc-emmen-v-az,0.411765
293,malm%C3%B6-ff-vs-ifk-norrk%C3%B6ping,kalmar-ff-v-bk-hacken,0.421053
1057,queens-park-rangers-vs-wigan-athletic,qpr-v-wigan,0.458333
80,az-alkmaar-vs-nec-nijmegen,az-v-nec,0.470588
272,psv-eindhoven-vs-rkc-waalwijk,psv-v-rkc,0.473684
574,ankarag%C3%BCc%C3%BC-vs-istanbul-ba%C5%9Fak%C5...,ankaragucu-v-basaksehir,0.500000


In [14]:
df = df.drop(remove)
df = df.reset_index(drop = True)

In [ ]:
scores = []
for i in range(0, len(df['url']), 50):
    x = list(df['url'][i:i + 50])
    print('...')
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)
    time.sleep(2)

...


In [37]:
len(scores)

1155

In [43]:
df['target'] = scores
df = df.drop(['url','odds_url'], axis = 1)
df = df[df['target'] != 'remove']

In [44]:
df.to_csv('ml_df.csv')

In [45]:
df

,expected_goals_(xg)_diff,total_shots_diff,big_chances_diff,big_chances_missed_diff,accurate_passes_diff,accurate_passes_percentage_diff,fouls_committed_diff,offsides_diff,corners_diff,shots_off_target_diff,shots_on_target_diff,blocked_shots_diff,hit_woodwork_diff,shots_inside_box_diff,shots_outside_box_diff,xg_open_play_diff,xg_set_play_diff,xg_on_target_(xgot)_diff,passes_diff,own_half_diff,opposition_half_diff,accurate_long_balls_diff,accurate_long_balls_percentage_diff,accurate_crosses_diff,accurate_crosses_percentage_diff,throws_diff,yellow_cards_diff,red_cards_diff,tackles_won_diff,tackles_won_percentage_diff,interceptions_diff,blocks_diff,clearances_diff,keeper_saves_diff,duels_won_diff,ground_duels_won_diff,ground_duels_won_percentage_diff,aerial_duels_won_diff,aerial_duels_won_percentage_diff,successful_dribbles_diff,successful_dribbles_percentage_diff,score_diff,posession_diff,odds_sum,xg_penalty_diff,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,cv_odds,target
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,1.091726,NaN,0,0.057012,0.039078,0.166599,2.300000,2.387500,4.200000,0.362059,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,1.087449,NaN,2,0.191021,0.863459,0.016510,4.750000,13.000000,1.250000,0.952563,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,1.044048,NaN,2,2.616801,18.683692,0.004381,20.000000,56.000000,1.024390,1.087591,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,1.06456,NaN,2,0.066525,0.227323,0.016281,3.500000,6.500000,1.600000,0.638920,1
4,-0.95,1.0,-2.0,0.0,-21.0,-0.02,0.0,1.0,2.0,2.0,-1.0,0.0,0.0,0.0,1.0,-0.04,-0.13,-1.99,-19.0,-59.0,38.0,-1.0,0.20,0.0,-0.04,5.0,0.0,0.0,-4.0,-0.28,4.0,0.0,-4.0,0.0,-5.0,-6.0,-0.26,1.0,0.05,0.0,-0.25,-1.0,-4.0,1.092484,-0.78,2,0.121344,0.143119,0.041334,3.126923,4.500000,1.816667,0.426254,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,0.74,1.0,2.0,1.0,-88.0,-0.09,2.0,0.0,-1.0,2.0,0.0,-1.0,0.0,3.0,-2.0,0.76,-0.02,0.68,-87.0,-34.0,-54.0,5.0,0.01,2.0,0.15,-1.0,-1.0,0.0,0.0,0.40,-2.0,1.0,12.0,1.0,8.0,-3.0,-0.17,11.0,0.74,1.0,0.33,1.0,-18.0,1.089149,NaN,1,0.112699,0.019385,0.220418,3.400000,1.533333,7.000000,0.698570,0
1150,-0.08,-2.0,0.0,-1.0,-9.0,-0.03,-1.0,0.0,-1.0,-3.0,0.0,1.0,0.0,-2.0,0.0,-0.33,0.25,-0.20,0.0,23.0,-32.0,-5.0,-0.06,-1.0,0.03,-1.0,-1.0,0.0,-5.0,-0.17,0.0,-1.0,2.0,0.0,-13.0,-10.0,-0.38,-3.0,-0.34,-3.0,-0.33,0.0,0.0,1.054593,NaN,0,0.055762,0.067525,0.104070,2.375000,2.875000,3.500000,0.193254,0
1151,-0.08,0.0,0.0,0.0,26.0,0.04,0.0,0.0,3.0,-1.0,1.0,0.0,0.0,0.0,0.0,-0.06,-0.02,-0.34,21.0,-32.0,58.0,2.0,0.28,3.0,0.25,1.0,1.0,0.0,-2.0,-0.19,2.0,0.0,-4.0,-2.0,9.0,3.0,0.14,6.0,0.42,3.0,0.43,-1.0,6.0,1.065775,NaN,2,0.071807,0.077802,0.059512,3.000000,2.875000,2.600000,0.072437,1
1153,0.53,2.0,2.0,1.0,18.0,0.04,-1.0,1.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,0.53,NaN,0.69,12.0,44.0,-26.0,7.0,0.21,0.0,0.09,0.0,-1.0,0.0,0.0,0.10,-3.0,0.0,-3.0,-1.0,5.0,0.0,0.00,5.0,0.56,0.0,-0.10,1.0,0.0,1.086078,NaN,1,0.210004,0.019400,0.785674,4.600000,1.285714,11.000000,0.877336,1
